In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os
import sys
import traceback

# disable pycache
sys.dont_write_bytecode = True


In [2]:
from pudu.app.main import App

In [3]:
from pudu.app.main import *

In [4]:
config_paths = [
            'database_config.yaml',
            '../src/pudu/configs/database_config.yaml',
            'src/pudu/configs/database_config.yaml',
            'pudu/configs/database_config.yaml',
            '/opt/database_config.yaml'
]

config_path = None
for path in config_paths:
    if os.path.exists(path):
        config_path = path
        break

In [5]:
config = DatabaseConfig(config_path)
task_tables = initialize_tables_from_config(config, 'robot_task')

In [6]:
table = task_tables[0]

In [7]:
df = table.query_data_as_df()
df.head()

/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/rds/utils.py:138: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(f"SELECT * FROM {table_name}", connection)


,id,robot_sn,task_name,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,...,duration,efficiency,remaining_time,consumption,water_consumption,progress,status,create_time,update_time,tenant_id
0,1,811064412050012,ISA 1st floor wet,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,...,93.0,358.32,2319.0,0.0,NaN,4.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
1,2,811064412050012,ISA 1st floor wet,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,...,27.0,274.68,0.0,0.0,NaN,0.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
2,3,811064412050012,ISA 1st floor wet,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,...,77.0,324.18,2247.0,0.0,NaN,2.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
3,4,811064412050012,3rd floor dry,Sweeping,Carpet Vacuum,Carpet Vacuuming,Standard,Medium,Off,Off,...,1091.0,696.50,747.0,17.2,NaN,59.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
4,5,811064412050012,4th floor dry,Sweeping,Carpet Vacuum,Carpet Vacuuming,Standard,Medium,Off,Off,...,1359.0,795.83,0.0,25.8,NaN,100.0,Task Ended,2025-03-26 03:37:12,2025-07-16 21:36:09,000000


In [8]:
orig_data = df.to_dict(orient='records')
orig_data[0]

{'id': 1,
 'robot_sn': '811064412050012',
 'task_name': 'ISA 1st floor wet',
 'mode': 'Scrubbing',
 'sub_mode': 'Custom',
 'type': 'Custom',
 'vacuum_speed': 'Off',
 'vacuum_suction': 'Off',
 'wash_speed': 'Standard',
 'wash_suction': 'Medium',
 'wash_water': 'Medium',
 'map_name': '1#11#USF-ISA-1ST-FLOORV2',
 'map_url': 'https://fr-tech-cloud-open.s3.eu-central-1.amazonaws.com/pudu_cloud_platform/map/B00C9D5916E8/76db4733adaa537fdb1db23a2594910c.png',
 'actual_area': 9.26,
 'plan_area': 229.75,
 'start_time': Timestamp('2024-09-19 21:36:33'),
 'end_time': Timestamp('2024-09-19 21:39:49'),
 'duration': 93.0,
 'efficiency': 358.32,
 'remaining_time': 2319.0,
 'consumption': 0.0,
 'water_consumption': nan,
 'progress': 4.0,
 'status': 'Task Cancelled',
 'create_time': Timestamp('2025-03-26 03:37:12'),
 'update_time': Timestamp('2025-07-23 18:16:32'),
 'tenant_id': '000000'}

In [9]:
# case 1: new record
new_data = orig_data[0]
new_data['task_name'] = 'Re-clean again'
new_data['start_time'] = datetime.now()
new_data['end_time'] = datetime.now() + timedelta(hours=1)
new_data = pd.DataFrame([new_data])
new_data

,id,robot_sn,task_name,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,...,duration,efficiency,remaining_time,consumption,water_consumption,progress,status,create_time,update_time,tenant_id
0,1,811064412050012,Re-clean again,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,...,93.0,358.32,2319.0,0.0,NaN,4.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000


In [10]:
successful, failed, changes = insert_to_databases_with_change_detection(new_data, [table], 'robot_task')
changes

{'811064412050012_Re-clean again_2025-07-30 12:02:50.534969': {'robot_id': '811064412050012',
  'primary_key_values': {'robot_sn': '811064412050012',
   'task_name': 'Re-clean again',
   'start_time': Timestamp('2025-07-30 12:02:50.534969')},
  'change_type': 'new_record',
  'changed_fields': ['id',
   'robot_sn',
   'task_name',
   'mode',
   'sub_mode',
   'type',
   'vacuum_speed',
   'vacuum_suction',
   'wash_speed',
   'wash_suction',
   'wash_water',
   'map_name',
   'map_url',
   'actual_area',
   'plan_area',
   'start_time',
   'end_time',
   'duration',
   'efficiency',
   'remaining_time',
   'consumption',
   'water_consumption',
   'progress',
   'status',
   'create_time',
   'update_time',
   'tenant_id'],
  'old_values': {},
  'new_values': {'id': 1,
   'robot_sn': '811064412050012',
   'task_name': 'Re-clean again',
   'mode': 'Scrubbing',
   'sub_mode': 'Custom',
   'type': 'Custom',
   'vacuum_speed': 'Off',
   'vacuum_suction': 'Off',
   'wash_speed': 'Standard',


In [11]:
notification_service = NotificationService()

In [12]:
notif_success, notif_failed = send_change_based_notifications(
                    notification_service, changes, 'robot_task'
                )

title: Re-clean again, content: Robot: Robot 811064412050012; Task Re-clean again's status is Task Cancelled., severity: warning, status: uncompleted
title: 🟠 Re-clean again 🚫, content: Robot: Robot 811064412050012; Task Re-clean again's status is Task Cancelled., severity: warning, status: uncompleted
